# SMILES 처리 방법

### 1. 분자 지문 (Molecular Fingerprints)
1. 데이터 처리: SMILES로부터 분자 지문을 추출
2. 모델 학습: 예를 들어 RandomForestRegressor 사용
3. 하이퍼파라미터 최적화: GridSearchCV 또는 RandomizedSearchCV 사용

- 장점: 빠르고, 이해하기 쉽고, 다양한 머신러닝 알고리즘에 적용하기 쉽다
- 단점: 정보의 손실이 있을 수 있고, 분자 구조의 모든 세부 정보를 담기 어렵다

### 2. Mol2Vec
1. 데이터 처리: SMILES의 각 부분에 대한 임베딩을 생성
2. 모델 학습: 예를 들어, RandomForestRegressor 사용
3. 하이퍼파라미터 최적화

- 장점: 더 복잡한 패턴을 포착할 수 있다
- 단점: 학습 시간이 오래 걸릴 수 있고, 해석하기 어려울 수 있다

### 3. 그래프 기반 접근법
1. 데이터 처리: 분자를 그래프로 변환
2. 모델 학습: Graph Neural Networks (GNNs) 사용
3. 하이퍼파라미터 최적화

- 장점: 분자 구조의 복잡성을 더 잘 모델링할 수 있다
- 단점: 계산 비용이 높고, 구현이 복잡할 수 있다

### 4. One-hot Encoding
1. 데이터 처리: SMILES 문자열의 각 문자를 one-hot 벡터로 인코딩
2. 모델 학습: RandomForestRegressor 사용
3. 하이퍼파라미터 최적화

- 장점: 간단하고 빠르다
- 단점: SMILES 문자열의 길이가 다르면 패딩이 필요하고, 문자열의 길이가 길 경우 차원이 높아진다

### 5. Sequence Models (RNN, LSTM, GRU 등)
1. 데이터 처리: SMILES 문자열을 시퀀스로 변환
2. 모델 학습: LSTM 또는 GRU 사용
3. 하이퍼파라미터 최적화

- 장점: 시퀀스 데이터의 내재된 의미를 잘 포착할 수 있다
- 단점: 학습 시간이 오래 걸리고, 복잡한 모델 구조를 필요로 한다

### 6. Transformer-based models
1. 데이터 처리: SMILES 문자열을 시퀀스로 변환
2. 모델 학습: Transformer architecture (e.g., BERT for chemistry) 사용
3. 하이퍼파라미터 최적화

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Flatten

# 데이터 로드
train_data = pd.read_csv('train.csv')


## 1. 분자 지문 (Molecular Fingerprints)


In [2]:
def generate_fingerprint(smiles_string):
    mol = Chem.MolFromSmiles(smiles_string)
    fingerprint = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048)
    return list(fingerprint)

# Fingerprints 생성
train_data['fingerprint'] = train_data['SMILES'].apply(generate_fingerprint)
X = list(train_data['fingerprint'])
y_HLM = train_data['HLM']
y_MLM = train_data['MLM']

# 모델 학습 및 평가
X_train, X_test, y_train_HLM, y_test_HLM = train_test_split(X, y_HLM, test_size=0.2, random_state=42)
X_train, X_test, y_train_MLM, y_test_MLM = train_test_split(X, y_MLM, test_size=0.2, random_state=42)

model_HLM = LinearRegression().fit(X_train, y_train_HLM)
model_MLM = LinearRegression().fit(X_train, y_train_MLM)

y_pred_HLM = model_HLM.predict(X_test)
y_pred_MLM = model_MLM.predict(X_test)

mse_HLM = mean_squared_error(y_test_HLM, y_pred_HLM)
mse_MLM = mean_squared_error(y_test_MLM, y_pred_MLM)
rmse_HLM = np.sqrt(mse_HLM)
rmse_MLM = np.sqrt(mse_MLM)
print("RMSE for HLM using Fingerprints:", rmse_HLM)
print("RMSE for MLM using Fingerprints:", rmse_MLM)

RMSE for HLM using Fingerprints: 66.60611383280072
RMSE for MLM using Fingerprints: 64.16422717594158


In [3]:
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
import numpy as np
import pandas as pd

# 데이터 불러오기
train_data = pd.read_csv('./submissions/train_linear.csv')
test_data = pd.read_csv('./submissions/test_linear.csv')

# SMILES 문자열에서 fingerprint를 생성하는 함수
def generate_fingerprint(smiles_string):
    mol = Chem.MolFromSmiles(smiles_string)
    if mol is not None:
        fingerprint = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048)
        return np.array(fingerprint)
    else:
        return np.zeros(2048)

# Train 데이터에 대해 fingerprints 생성
fingerprint_data = np.array(train_data['SMILES'].apply(generate_fingerprint).tolist())
fingerprint_df = pd.DataFrame(fingerprint_data, columns=[f'bit_{i}' for i in range(2048)])
train_data_expanded = pd.concat([train_data, fingerprint_df], axis=1)

# Test 데이터에 대해 fingerprints 생성 (필요한 경우)
fingerprint_data_test = np.array(test_data['SMILES'].apply(generate_fingerprint).tolist())
fingerprint_df_test = pd.DataFrame(fingerprint_data_test, columns=[f'bit_{i}' for i in range(2048)])
test_data_expanded = pd.concat([test_data, fingerprint_df_test], axis=1)

def train_and_evaluate(target_column):
    # 훈련 데이터 준비
    X = train_data_expanded.drop(columns=['id', 'SMILES', 'MLM', 'HLM'])
    y = train_data_expanded[target_column]

    # 데이터 분할
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # 모델 학습
    model = RandomForestRegressor()
    param_grid = {
        'n_estimators': [10, 50, 100, 200],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }
    grid_search = GridSearchCV(model, param_grid, cv=3, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)
    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_

    # RMSE 계산
    predictions = best_model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, predictions))
    print(f"RMSE for {target_column}:", rmse)
    
    return best_model

# 각각의 타겟(MLM, HLM)에 대해 학습 및 평가 수행
best_model_MLM = train_and_evaluate('MLM')
best_model_HLM = train_and_evaluate('HLM')


Fitting 3 folds for each of 144 candidates, totalling 432 fits
RMSE for MLM: 32.793645901928336
Fitting 3 folds for each of 144 candidates, totalling 432 fits
RMSE for HLM: 32.16512458417776


## 2. Mol2Vec

In [15]:
from mol2vec.features import mol2alt_sentence, MolSentence, DfVec, sentences2vec
from rdkit import Chem
from gensim.models import Word2Vec

# 1. 모델 로드
model = Word2Vec.load('model_300dim.pkl')  # 실제 경로로 변경 필요

# 2. 데이터 전처리
train_data = pd.read_csv('./submissions/train_linear.csv')
train_data['sentence'] = train_data.apply(lambda x: MolSentence(mol2alt_sentence(Chem.MolFromSmiles(x['SMILES']), 1)), axis=1)

# 3. 임베딩 생성
train_data['mol2vec'] = [DfVec(x) for x in sentences2vec(train_data['sentence'], model, unseen='UNK')]

# 이후에는 임베딩된 데이터를 사용하여 예측 모델을 학습하거나 다른 분석을 수행할 수 있습니다.


UnpicklingError: invalid load key, '{'.

In [13]:
from mol2vec.features import mol2alt_sentence
from rdkit import Chem
from gensim.models import Word2Vec

# 1. 데이터 전처리
train_data = pd.read_csv('./submissions/train_linear.csv')
sentences = [mol2alt_sentence(Chem.MolFromSmiles(smiles), 1) for smiles in train_data['SMILES']]

# 2. Word2Vec 학습
model = Word2Vec(sentences, vector_size=300, window=10, min_count=1, workers=4)
model.save("mol2vec_model.pkl")

# 이후에는 학습된 모델을 불러와서 필요한 변환을 수행할 수 있습니다.
